---
title : "Overstromingsrisico scenarios"
execute:
    output: asis
---

Scenarios worden gebruikt om de mogelijke gevolgen van overstromingen in kaart te brengen. <br>
Hiermee kan de kans uit een fragilitycruve gecombineerd worden met de gevolgen die bepaald zijn bij een scenario om zo een beter inzicht te krijgen in het totale risico. <br>
Dit kan helpen om prioriteiten te stellen op basis van een totaal beeld en niet alleen op basis van kans of gevolg afzonderlijk. <br>
Om de scenarios te kunnen combineren moeten er een aantal stappen worden doorlopen:
- Omrekenen van dijkvakkansen naar trajectdeelkansen ([CalculateFloodScenarioProbability](#sec-CalculateFloodScenarioProbability))
- Belasting per deeltraject bepalen ([LoadFromFloodScenarioProbabilit](#sec-LoadFromFloodScenarioProbabilit))
- Met de belasting een bijpassende overstromingscenario selecteren ([SelectFloodScenarioFromLoad ](#sec-SelectFloodScenarioFromLoad))
- Van het scenario en de curve een geagregeerd risico bepalen ([CalculateFloodRisk](#sec-CalculateFloodRisk))


In [ ]:
from pathlib import Path

from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter

In [ ]:
path = Path.cwd() / "data_sets" / "7.flood_scenarios"
config = Config(config_path=path / "test_calculate_flood_scenario_probability.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
from toolbox_continu_inzicht.flood_scenarios import CalculateFloodScenarioProbability

## CalculateFloodScenarioProbability{sec-CalculateFloodScenarioProbability}

Dit voorbeeld laat zien hoe met behulp van de `CalculateFloodScenarioProbability`  dijkvakkansen naar trajectdeelkansen kunnen worden omgezet. <br>
Hiervoor is de gecombineerde faalkans per dijkvak nodig en een mapping van dijkvakken naar dijktraject delen van daar naar de overstromingsscenario gebieden.
```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [-24,0,24,48]

DataAdapter:
    default_options:
        csv:
            sep: ","
    combined_failure_probability_data:
        type: csv
        file: "combined_failure_probability_data.csv"
    section_id_to_segment_id:
        type: csv
        file: "section_id_to_segment_id.csv"
```

In [ ]:
calculate_flood_scenario_probability = CalculateFloodScenarioProbability(
    data_adapter=data_adapter
)
calculate_flood_scenario_probability.run(
    input=["combined_failure_probability_data", "section_id_to_segment_id"],
    output="segment_failure_probability",
)
# todo: segment to dikesystem

In [ ]:
calculate_flood_scenario_probability.df_in_sections_to_segment

In [ ]:
calculate_flood_scenario_probability.df_in_grouped_sections_failure_probability

In [ ]:
calculate_flood_scenario_probability.df_out